# Terrain-following coordinates, again

In the first notebook on terrain-following coordinates, we look at diffusion and advection-diffusion problems.
Now let's try something harder: viscous fluid flow.
Before, we had to transform vector fields and gradients of scalar fields.
Here our solution variable is a vector field and we'll need to take its (symmetric) gradient, so there are a lot more metric terms.
To review, the terrain-following coordinate transformation is
$$\begin{align}
\xi_1 & = x_1 \\
\xi_2 & = x_2 \\
\xi_3 & = H\frac{x_3 - b(x_1, x_2)}{h(x_1, x_2)}
\end{align}$$
which, upon defining the geometric distortion factor
$$\gamma = -\frac{H}{h}\left(\nabla b + \xi_3\nabla h\right)$$
has the Jacobian
$$\frac{d\xi}{dx} = \left[\begin{matrix}I & 0 \\ \gamma^\top & H / h\end{matrix}\right].$$
We'll write $u_x$ and $u_\xi$ for the vector fields in Cartesian and terrain-following coordinates respectively.

In [ ]:
import firedrake
from firedrake import Constant, inner, sym, grad, div, dot
import numpy as np
import matplotlib.pyplot as plt

nx, nz = 32, 32
computational_domain = firedrake.UnitSquareMesh(nx, nz, diagonal="crossed")

In [ ]:
δb = Constant(1 / 4)
α = Constant(1 / 8)
ξ_0 = Constant(1 / 2)

def sech(z):
    return 2 / (firedrake.exp(z) + firedrake.exp(-z))

def bed_topography(ξ):
    return δb * sech((ξ - ξ_0) / α)

In [ ]:
ξ = firedrake.SpatialCoordinate(computational_domain)
bed_expr = bed_topography(ξ[0])
surf_expr = Constant(1.0)
expr = firedrake.as_vector((ξ[0], (1 - ξ[1]) * bed_expr + ξ[1] * surf_expr))

Vc = computational_domain.coordinates.function_space()
X = firedrake.Function(Vc)
X.interpolate(expr)
physical_domain = firedrake.Mesh(X, reorder=False)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
firedrake.triplot(physical_domain, axes=ax)
ax.legend();

### Computing strain rates

Before we can really do anything, we need to know how to compute strain rates in both reference frames.
In Cartesian coordinates this is the same as it's always been.

In [ ]:
def cartesian_strain_rate(u):
    return sym(grad(u))

In terrain-following coordinates, we have to think about how all the fields get warped again.
First, recall that
$$u_x = J^{-1}u_\xi,$$
which we showed in the previous notebook from the chain rule.

But next we have to ask how the gradient of $u$ transforms.
When we type `grad(u)` and `u` is in a vector function space, we get a tensor.
What do the indices of this tensor mean?
Do the rows correspond to vector components of `u` and the columns correspond to what spatial variable we're differentiating, or the other way around?
If you're not sure, you can check this -- make a vector field equal to $(x, 0)^\top$, then $(y, 0)^\top$, and repeat for the other component, evaluate their derivatives, project them into a tensor function space, and check the result at a point.
I already did this so I can tell you that as far as Firedrake is concerned,
$$\text{grad}(u)_{ij} = \frac{\partial u_i}{\partial x_j},$$
i.e. vector components along rows and variables along columns.
This means that to transform the gradient of $u$, we multiply on the right by the Jacobian of the coordinate transformation:
$$\nabla_xu_x = \nabla_\xi(J^{-1}u_\xi)J$$
and we can symmetrize it to get the strain rate tensor:
$$\begin{align}
\dot\varepsilon_x(u_x) & = \frac{1}{2}\left(\nabla_xu_x + \nabla_xu_x^\top\right) \\
& = \frac{1}{2}\left\{\nabla_\xi(J^{-1}u_\xi)J + J^*\nabla_\xi(J^{-1}u_\xi)^*\right\}
\end{align}$$

In [ ]:
ξ = firedrake.SpatialCoordinate(computational_domain)
b = bed_topography(ξ[0])
h = surf_expr - b
H = Constant(1.0)

γ = -H * (b.dx(0) + ξ[1] * h.dx(0)) / h
dξ_dx = firedrake.as_tensor(
    [
        [1.0, 0.0],
        [γ, H / h],
    ]
)

η = b.dx(0) + ξ[1] * h.dx(0) / H
dx_dξ = firedrake.as_tensor(
    [
        [1.0, 0.0],
        [η, h / H]
    ]
)

def terrain_following_strain_rate(u):
    u_x = dot(dx_dξ, u)
    grad_u = dot(grad(u_x), dξ_dx)
    return sym(grad_u)

### The solver

The setup here is that we're computing a velocity field that solves the Stokes equations but with friction along the top surface.
We'll write $\kappa$ for the friction coefficient.
First we'll write a generic procedure to solve the Stokes problem that will work for both the Cartesian and terrain-following cases.
The variational form is
$$\int_\Omega\left\{2g\mu\dot\varepsilon(u):\dot\varepsilon(v) - p\nabla\cdot gv - q\nabla\cdot gu\right\}dx + \int_\Gamma \kappa(u - u_\Gamma)\cdot v\,d\gamma = 0$$
together with the zero normal flow boundary condition at the top surface, $u\cdot\nu|_\Gamma = 0$.
Here $g$ is again the volume distortion factor, which is 1 in Cartesian coordinates, and equal to $h / H$ in terrain-following coordinates.

In [ ]:
top_ids = (4,)
side_ids = (1, 2, 3)

def solve_stokes(Z, μ, κ, g, ε, degree=8):
    z = firedrake.Function(Z)
    u, p = firedrake.split(z)
    v, q = firedrake.TestFunctions(Z)

    kw = {"degree": degree}
    dx = firedrake.dx(**kw)
    dγ = firedrake.ds(top_ids, **kw)
    F_interior = (2 * g * μ * inner(ε(u), ε(v)) - p * div(g * v) - q * div(g * u)) * dx
    F_boundary = κ * inner(u - u_Γ, v) * dγ
    F = F_interior + F_boundary

    bc_top = firedrake.DirichletBC(Z.sub(0).sub(1), Constant(0.0), top_ids)
    bc_sides = firedrake.DirichletBC(Z.sub(0), Constant((0.0, 0.0)), side_ids)

    basis = firedrake.VectorSpaceBasis(constant=True, comm=firedrake.COMM_WORLD)
    nullspace = firedrake.MixedVectorSpaceBasis(Z, [Z.sub(0), basis])
    firedrake.solve(F == 0, z, [bc_top, bc_sides], nullspace=nullspace)
    return z

### The solutions

Now let's actually compute the velocities and compare.

In [ ]:
cg2 = firedrake.FiniteElement("CG", "triangle", 2)
cg1 = firedrake.FiniteElement("CG", "triangle", 1)

Q_physical = firedrake.FunctionSpace(physical_domain, cg1)
V_physical = firedrake.VectorFunctionSpace(physical_domain, cg2)
Z_physical = V_physical * Q_physical

Q_computational = firedrake.FunctionSpace(computational_domain, cg1)
V_computational = firedrake.VectorFunctionSpace(computational_domain, cg2)
Z_computational = V_computational * Q_computational

In [ ]:
μ = Constant(1.0)
λ = Constant(0.1)
κ = Constant(μ / λ)
u_Γ = Constant((1.0, 0.0))

In [ ]:
z_physical = solve_stokes(Z_physical, μ, κ, Constant(1.0), cartesian_strain_rate)
z_terrain = solve_stokes(Z_computational, μ, κ, h / H, terrain_following_strain_rate)

u_physical, p_physical = z_physical.subfunctions
u_terrain, p_terrain = z_terrain.subfunctions

### Comparison

In order to compare the two solutions, I'll have to remap the terrain-following solution back into Cartesian coordinates.
This follows again the formula $u_x = J^{-1}u_\xi$.
Ask me about the bug that I made the first time I wrote this.

In [ ]:
x = firedrake.SpatialCoordinate(physical_domain)
b = bed_topography(x[0])
h = surf_expr - b
H = Constant(1.0)

η = b.dx(0) + (x[1] - b) / h * h.dx(0)
dx_dξ = firedrake.as_tensor(
    [
        [1.0, 0.0],
        [η, h / H]
    ]
)

u_transfer = firedrake.Function(V_physical)
u_transfer.dat.data[:] = u_terrain.dat.data_ro[:]
u_transfer.interpolate(dot(dx_dξ, u_transfer));

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.streamplot(u_physical, resolution=1.0/(2 * nx), seed=1729, axes=ax)
fig.colorbar(colors);

When we plot the norm difference between the two solutions, we find that it's less than .2% of the total speed.
The discrepancy is due to differences in integrating the geometric distortion factors.
If we run this again at finer resolution, we should see the discrepancy go down.

In [ ]:
δu = firedrake.Function(V_physical).interpolate(u_transfer - u_physical)

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
colors = firedrake.tripcolor(δu, axes=ax)
fig.colorbar(colors);